In [ ]:
from dataset_loader.dataset_loader import dataset_loader


_, _, test_data_p = dataset_loader('cora')

In [5]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="dataset", name="Cora", split="full")
dataset = dataset[0]  # there is only one graph in the dataset

test_data = dataset.subgraph(dataset.test_mask)

for i in range(len(test_data.y)):
	print(test_data.y[i], test_data_p.y[i])

tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(0) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(1) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2

In [ ]:
from torch_geometric.datasets import Planetoid
import os
from torch_geometric.loader import DataLoader
import torch_geometric
import torch
import torch.nn.functional as F
from dataset_loader.dataset_loader import dataset_loader


DATASET_STORAGE_PATH = "./dataset/"

train_data, val_data, test_data = dataset_loader('cora')



In [16]:
# count the how many elements are for each class (are 1-hot encoded)
print(test_data.y[:,0].sum())
print(test_data.y[:,1].sum())
print(test_data.y[:,2].sum())
print(test_data.y[:,3].sum())


tensor(149.)
tensor(103.)
tensor(64.)
tensor(684.)


In [5]:
from utils.math import interval_softmax, actually_reachable
import torch
import numpy as np

a_L = np.array([9.6062404e-06, 9.9997389e-01, 1.6473108e-05])
a_U = np.array([9.6254726e-06, 9.9997395e-01, 1.6506086e-05])

assert np.all(a_L <= a_U), "Lower bound should be less than upper bound"

q_L, q_U = actually_reachable(a_L, a_U)

assert np.all(q_L <= q_U), "Lower bound should be less than upper bound"

print("Lower bound: ", q_L)
print("Upper bound: ", q_U)

Lower bound:  [9.6062404e-06 9.9997389e-01 1.6473108e-05]
Upper bound:  [9.62547260e-06 9.99973921e-01 1.65037596e-05]


In [ ]:
from dataset_loader.dataset_loader import dataset_loader

train_loader, val_loader, test_loader = dataset_loader('ogb-arxiv-year', {"batch_size":1})

/vast/m.tolloso/miniconda3/envs/gu/lib/python3.11/site-packages/ogb/nodeproppred/dataset.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dict = torch.load(pre_pr

In [6]:
from dataset_loader.dataset_loader import dataset_loader

train_loader, val_loader, test_loader = dataset_loader('cora', {"batch_size":1})


In [14]:
for data in test_loader:
	print(data.y[data.y.sum(dim=1) == 0].shape)
	break

torch.Size([684, 3])


In [16]:
import gdown
dataset_drive_url = {
    # 'twitch-gamer_feat' : '1fA9VIIEI8N0L27MSQfcBzJgRQLvSbrvR',
    # 'twitch-gamer_edges' : '1XLETC6dG3lVl7kDmytEJ52hvDMVdxnZ0',
    'snap-patents' : '1ldh23TSY1PwXia6dU0MYcpyEgX-w3Hia', 
    'pokec' : '1dNs5E7BrWJbgcHeQ_zuy5Ozp2tRCWG0y', 
    'yelp-chi': '1fAXtTVQS4CfEk4asqrFw9EPmlUPGbGtJ', 
    'wiki_views': '1p5DlVHrnFgYm3VsNIzahSsvCD424AyvP', # Wiki 1.9M 
    'wiki_edges': '14X7FlkjrlUgmnsYtPwdh-gGuFla4yb5u', # Wiki 1.9M 
    'wiki_features': '1ySNspxbK-snNoAZM7oxiWGvOnTRdSyEK' # Wiki 1.9M
}

for key, value in dataset_drive_url.items():
	gdown.download(id=value, \
		output=f'{DATASET_STORAGE_PATH}/{key}.mat', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ldh23TSY1PwXia6dU0MYcpyEgX-w3Hia
From (redirected): https://drive.google.com/uc?id=1ldh23TSY1PwXia6dU0MYcpyEgX-w3Hia&confirm=t&uuid=ba1c433d-e201-4df7-96bc-2bde139b2938
To: /vast/m.tolloso/graph-uncertainty/dataset/snap-patents.mat
100%|██████████| 320M/320M [00:04<00:00, 69.3MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1dNs5E7BrWJbgcHeQ_zuy5Ozp2tRCWG0y
From (redirected): https://drive.google.com/uc?id=1dNs5E7BrWJbgcHeQ_zuy5Ozp2tRCWG0y&confirm=t&uuid=52b85cec-d9cc-40e4-81ea-239e0886a23b
To: /vast/m.tolloso/graph-uncertainty/dataset/pokec.mat
100%|██████████| 1.35G/1.35G [00:35<00:00, 38.2MB/s]


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1fAXtTVQS4CfEk4asqrFw9EPmlUPGbGtJ

but Gdown can't. Please check connections and permissions.

In [3]:
for batch in train_loader:
	print(batch)
	print(batch.y[:,0].sum())
	print(batch.y[:,1].sum())
	print(batch.y[:,2].sum())
	print(len(batch.y[batch.y.sum(dim=1) == 0]))
	break

DataBatch(x=[647, 2325], edge_index=[2, 3763], y=[647, 3], train_mask=[647, 10], val_mask=[647, 10], test_mask=[647, 10], batch=[647], ptr=[2])
tensor(218.)
tensor(241.)
tensor(188.)
0


In [5]:
stringa = "1fA9VIIEI8N0L27MSQfcBzJgRQLvSbrvR"

"1fA9VIIEI8N0L27MSQfcBzJgRQLvSbrvR" in stringa.lower()

False